In [14]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

import SIMBBAAT_fct as smbt

import ipywidgets as widgets
import ipyvuetify as v

from datetime import datetime
import webbrowser
import requests
import tempfile

GitHub_repo_url = 'https://github.com/mluyat/SIMBBAAT'
GitHub_repo_python_url = 'https://github.com/mluyat/SIMBBAAT/blob/main/SIMBBAAT_fct.py'
TI_url = 'https://www.techniques-ingenieur.fr/base-documentaire/ingenierie-des-transports-th14/structures-et-materiaux-pour-l-aeronautique-42632210/solutions-analytiques-pour-l-assemblage-colle-en-simple-cisaillement-trp4038/'

In [15]:
# App bar buttons definition
GitHub_button = v.Btn(children=[v.Icon(children=['mdi-github-circle'])], href=GitHub_repo_url, target='_blank', style_='width: 70px;')
Python_button = v.Btn(children=[v.Icon(children=['mdi-language-python'])], href=GitHub_repo_python_url, target='_blank', class_='mx-1', style_='width: 70px;')
TI_button = v.Btn(children=[v.Icon(children=['mdi-file-outline'])], href=TI_url, target='_blank', style_='width: 70px;')


# Menu bar definition
Menu_bar = v.AppBar(children=[
        v.ToolbarTitle(children=['SIMBBAAT'], style_='font-size: 32px; font-weight: bold;'),
        v.Spacer(),
        GitHub_button,
        Python_button,
        TI_button],
        style_='background-color: #f6f8fa;')


# User input buttons definition
GR_active_button = v.Btn(v_model=False,children=['Goland & Reissner'], class_='flex-grow-1',outlined=True,style_='background-color: #F0F0F0; color: black; width: 100px;')
GR_active_button.kwargs={'id':'GR','active_color':'#0072BD'}
GR_plot_param_button = v.Btn(children=[v.Icon(children=['mdi-palette'])], class_='ml-1', style_='width: 70px;', outlined=True)

HS_active_button = v.Btn(v_model=False,children=['Hart-Smith'], class_='flex-grow-1',outlined=True,style_='background-color: #F0F0F0; color: black;')
HS_active_button.kwargs={'active_color':'#D95319'}
HS_plot_param_button = v.Btn(children=[v.Icon(children=['mdi-palette'])], class_='ml-1', style_='width: 70px;', outlined=True)

OE_active_button = v.Btn(v_model=False,children=['Ojalvo and Eidinoff'], class_='flex-grow-1',outlined=True,style_='background-color: #F0F0F0; color: black;')
OE_active_button.kwargs={'active_color':'#EDB120'}
OE_plot_param_button = v.Btn(children=[v.Icon(children=['mdi-palette'])], class_='ml-1', style_='width: 70px;', outlined=True)

O_active_button = v.Btn(v_model=False,children=['Oplinger'], class_='flex-grow-1',outlined=True,style_='background-color: #F0F0F0; color: black;')
O_active_button.kwargs={'active_color':'#7E2F8E'}
O_plot_param_button = v.Btn(children=[v.Icon(children=['mdi-palette'])], class_='ml-1', style_='width: 70px;', outlined=True)

LT_active_button = v.Btn(v_model=False,children=['Luo & Tong'], class_='flex-grow-1',outlined=True,style_='background-color: #F0F0F0; color: black;')
LT_active_button.kwargs={'active_color':'#77AC30'}
LT_plot_param_button = v.Btn(children=[v.Icon(children=['mdi-palette'])], class_='ml-1', style_='width: 70px;', outlined=True)

Z_active_button = v.Btn(v_model=False,children=['Zaho'], class_='flex-grow-1',outlined=True,style_='background-color: #F0F0F0; color: black;')
Z_active_button.kwargs={'active_color':'#4DBEEE'}
Z_plot_param_button = v.Btn(children=[v.Icon(children=['mdi-palette'])], class_='ml-1', style_='width: 70px;', outlined=True)


# User input fields definition 
f_field = v.TextField(v_model=4000, label='Force', type='number', style_='max-width : 200px;')
L_field = v.TextField(v_model=12.5, label='Overlap length', type='number', style_='max-width : 200px;')
w_field = v.TextField(v_model=25, label='Overlap width', type='number', class_='mx-10', style_='max-width : 200px;')
lr_field = v.TextField(v_model=50, label='Free length', type='number', style_='max-width : 200px;')
er_field = v.TextField(v_model=1.6, label='Substrats thickness', type='number', style_='max-width : 200px;')
ec_field = v.TextField(v_model=0.2, label='Adhesive thickness', type='number', style_='max-width : 200px;')
Er_field = v.TextField(v_model=70000, label="Substrats Young's modulus", type='number', class_='mx-10', style_='max-width : 200px;')
Ec_field = v.TextField(v_model=2800, label="Adhesive Young's modulus", type='number', class_='mx-10', style_='max-width : 200px;')
nur_field = v.TextField(v_model=0.31, label="Substrats Poisson's ratio", type='number', style_='max-width : 200px;')
nuc_field = v.TextField(v_model=0.4, label="Adhesive Poisson's ratio", type='number', style_='max-width : 200px;')
kb_field = v.TextField(v_model=1, label='kb', type='number', style_='max-width : 200px;')
k_field = v.Select(v_model='GR', label='k', items=['GR','HS'], class_='mx-10', style_='max-width : 200px;')
npts_field = v.TextField(v_model=501, label='Number of query point', type='number', style_='max-width : 200px;')


# Pop-up elements definition
color_picker = v.ColorPicker(width=400)
save_button = v.Btn(v_model=False,children=['Save'], color='success', class_='d-flex flex-grow-1')
discard_button = v.Btn(v_model=False,children=['Discard'], color='error', class_='d-flex flex-grow-1')

In [16]:
# Callback functions definition
def on_field_change(change):
    field_list = [f_field.v_model, 
                  L_field.v_model, w_field.v_model, lr_field.v_model, 
                  er_field.v_model, ec_field.v_model, Er_field.v_model, Ec_field.v_model, nur_field.v_model, nuc_field.v_model, 
                  kb_field.v_model, k_field.v_model, 
                  npts_field.v_model]
    if not ('' in field_list): 
        fct_plot_display()

def on_model_button_click(widget, event, data):
    widget.v_model = not widget.v_model
    widget.outlined = not widget.outlined
    if widget.v_model:
        widget.style_ = 'background-color: '+widget.kwargs['active_color']+'; color: white;'
        fct_plot_display()
    else:
        widget.style_ = 'background-color: #F0F0F0; color: black;'
        fct_plot_display()
 
def on_save_change_click(button, widget, event, data):
    button.kwargs['active_color'] = color_picker.v_model
    if button.v_model:
        button.style_ = 'background-color: '+button.kwargs['active_color']+'; color: white;'
    else:
        button.style_ = 'background-color: #F0F0F0; color: black;'
    plot_param_popup.v_model = False
    fct_plot_display()
    

def on_plot_param_button_click(button, widget, event, data):
    color_picker.v_model=button.kwargs.get('active_color')
    plot_param_popup.v_model=True
    save_button.on_event('click', lambda widget, event, data: on_save_change_click(button, widget, event, data))
    discard_button.on_event('click', lambda *args: setattr(plot_param_popup, 'v_model', False))

def fetch_display_data(url):
    with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as temp_file:
        temp_file.write(requests.get(url).text)
        temp_file_path = temp_file.name
    webbrowser.open(temp_file_path)

def fct_plot_display():
    f = f_field.v_model
    L = float(L_field.v_model)
    w = float(w_field.v_model)
    lr = float(lr_field.v_model)
    er = float(er_field.v_model)
    ec = float(ec_field.v_model)
    Er = float(Er_field.v_model)
    Ec = float(Ec_field.v_model)
    nur = float(nur_field.v_model)
    nuc = float(nuc_field.v_model)
    kb = float(kb_field.v_model)
    k = k_field.v_model
    npts = int(npts_field.v_model)
    
    T_GR = smbt.Fct_T_GR(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)
    S_GR = smbt.Fct_S_GR(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)
    
    T_HS = smbt.Fct_T_HS(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts,kb)
    S_HS = smbt.Fct_S_HS(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts,kb)
    
    T_OE = smbt.Fct_T_OE(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts,kb,k)
    S_OE = smbt.Fct_S_OE(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts,kb,k)
    
    T_O = smbt.Fct_T_O(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)
    S_O = smbt.Fct_S_O(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)
    
    T_LT = smbt.Fct_T_LT(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)
    S_LT = smbt.Fct_S_LT(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)
    
    T_Z = smbt.Fct_T_Z(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)
    S_Z = smbt.Fct_S_Z(f,L,w,lr,er,ec,Er,Ec,nur,nuc,npts)

    with output:
        clear_output(wait=True)
        plt.figure(figsize=(20, 7))
    
        X = smbt.Fct_X(L, npts)
        X = X/max(X)
        
        # Plotting
        plt.subplot(221)
        if GR_active_button.v_model:
            plt.plot(X, T_GR, color=GR_active_button.kwargs['active_color'])
        if HS_active_button.v_model:
            plt.plot(X, T_HS, color=HS_active_button.kwargs['active_color'])
        if OE_active_button.v_model:
            plt.plot(X, T_OE, color=OE_active_button.kwargs['active_color'])
        if O_active_button.v_model:
            plt.plot(X, T_O, color=O_active_button.kwargs['active_color'])
        if LT_active_button.v_model:
            plt.plot(X, T_LT, color=LT_active_button.kwargs['active_color'])
        if Z_active_button.v_model:
            plt.plot(X, T_Z, color=Z_active_button.kwargs['active_color'])
        plt.title('T')
        plt.grid(True)
        plt.xlim([-1, +1])
        
        plt.subplot(222)
        max_list = []
        if GR_active_button.v_model:
            plt.plot(X, T_GR, color=GR_active_button.kwargs['active_color'])
            max_list.append(max(T_GR))
        if HS_active_button.v_model:
            plt.plot(X, T_HS, color=HS_active_button.kwargs['active_color'])
            max_list.append(max(T_HS))
        if OE_active_button.v_model:
            plt.plot(X, T_OE, color=OE_active_button.kwargs['active_color'])
            max_list.append(max(T_OE))
        if O_active_button.v_model:
            plt.plot(X, T_O, color=O_active_button.kwargs['active_color'])
            max_list.append(max(T_O))
        if LT_active_button.v_model:
            plt.plot(X, T_LT, color=LT_active_button.kwargs['active_color'])
            max_list.append(max(T_LT))
        if Z_active_button.v_model:
            plt.plot(X, T_Z, color=Z_active_button.kwargs['active_color'])
            max_list.append(max(T_Z))
        plt.title('T')
        plt.grid(True)
        plt.xlim([-1, -0.95])
        if len(max_list)!=0:
            plt.ylim([min(max_list)-5, max(max_list)+5])
        
        plt.subplot(223)
        if GR_active_button.v_model:
            plt.plot(X, S_GR, color=GR_active_button.kwargs['active_color'])
        if HS_active_button.v_model:
            plt.plot(X, S_HS, color=HS_active_button.kwargs['active_color'])
        if OE_active_button.v_model:
            plt.plot(X, S_OE, color=OE_active_button.kwargs['active_color'])
        if O_active_button.v_model:
            plt.plot(X, S_O, color=O_active_button.kwargs['active_color'])
        if LT_active_button.v_model:
            plt.plot(X, S_LT, color=LT_active_button.kwargs['active_color'])
        if Z_active_button.v_model:
            plt.plot(X, S_Z, color=Z_active_button.kwargs['active_color'])
        plt.title('S')
        plt.grid(True)
        plt.xlim([-1, +1])
        
        plt.subplot(224)
        max_list = []
        if GR_active_button.v_model:
            plt.plot(X, S_GR, color=GR_active_button.kwargs['active_color'])
            max_list.append(max(S_GR))
        if HS_active_button.v_model:
            plt.plot(X, S_HS, color=HS_active_button.kwargs['active_color'])
            max_list.append(max(S_HS))
        if OE_active_button.v_model:
            plt.plot(X, S_OE, color=OE_active_button.kwargs['active_color'])
            max_list.append(max(S_OE))
        if O_active_button.v_model:
            plt.plot(X, S_O, color=O_active_button.kwargs['active_color'])
            max_list.append(max(S_O))
        if LT_active_button.v_model:
            plt.plot(X, S_LT, color=LT_active_button.kwargs['active_color'])
            max_list.append(max(S_LT))
        if Z_active_button.v_model:
            plt.plot(X, S_Z, color=Z_active_button.kwargs['active_color'])
            max_list.append(max(S_Z))
        plt.title('S')
        plt.grid(True)
        plt.xlim([-1, -0.95])
        if len(max_list)!=0:
            plt.ylim([min(max_list)-5, max(max_list)+5])
        
    # Adjusting the layout
        plt.tight_layout()
        plt.show()

In [17]:
# Callback functions definition
f_field.observe(on_field_change, names='v_model')
L_field.observe(on_field_change, names='v_model')
w_field.observe(on_field_change, names='v_model')
lr_field.observe(on_field_change, names='v_model')
er_field.observe(on_field_change, names='v_model')
ec_field.observe(on_field_change, names='v_model')
Er_field.observe(on_field_change, names='v_model')
Ec_field.observe(on_field_change, names='v_model')
nur_field.observe(on_field_change, names='v_model')
nuc_field.observe(on_field_change, names='v_model')
kb_field.observe(on_field_change, names='v_model')
k_field.observe(on_field_change, names='v_model')
npts_field.observe(on_field_change, names='v_model')


# Model buttons callback
GR_active_button.on_event('click', on_model_button_click)
HS_active_button.on_event('click', on_model_button_click)
OE_active_button.on_event('click', on_model_button_click)
O_active_button.on_event('click', on_model_button_click)
LT_active_button.on_event('click', on_model_button_click)
Z_active_button.on_event('click', on_model_button_click)


# Plot param buttons callback
GR_plot_param_button.on_event('click', lambda widget, event, data: on_plot_param_button_click(GR_active_button, widget, event, data))
HS_plot_param_button.on_event('click', lambda widget, event, data: on_plot_param_button_click(HS_active_button, widget, event, data))
OE_plot_param_button.on_event('click', lambda widget, event, data: on_plot_param_button_click(OE_active_button, widget, event, data))
O_plot_param_button.on_event('click', lambda widget, event, data: on_plot_param_button_click(O_active_button, widget, event, data))
LT_plot_param_button.on_event('click', lambda widget, event, data: on_plot_param_button_click(LT_active_button, widget, event, data))
Z_plot_param_button.on_event('click', lambda widget, event, data: on_plot_param_button_click(Z_active_button, widget, event, data))

In [32]:
# Plot display definition
output = widgets.Output()
Plot_layout = v.Html(tag='div', style_='display: flex; justify-content: center; align-items: center; width: 100%;', children=[output])

# Buttons display definition
Buttons_Layout = v.Html(tag='div', class_='d-flex flex-column', style_='padding-top: 50px;', children=[
        v.Html(tag='div', class_='d-flex flex-row mb-2', children=[GR_active_button,GR_plot_param_button]),
        v.Html(tag='div', class_='d-flex flex-row my-2', children=[HS_active_button,HS_plot_param_button]),
        v.Html(tag='div', class_='d-flex flex-row my-2', children=[OE_active_button,OE_plot_param_button]),
        v.Html(tag='div', class_='d-flex flex-row my-2', children=[O_active_button,O_plot_param_button]),
        v.Html(tag='div', class_='d-flex flex-row my-2', children=[LT_active_button,LT_plot_param_button]),
        v.Html(tag='div', class_='d-flex flex-row mt-2', children=[Z_active_button,Z_plot_param_button]),
])


# Fields display definition
Parameter_fields_Layout = v.Html(tag='div', class_='d-flex flex-column', style_='padding-top: 50px;', children=[
    v.Html(tag='div', class_='d-flex flex-row justify-space-around', children=[L_field,w_field,lr_field]),
    v.Html(tag='div', class_='d-flex flex-row justify-space-around', children=[er_field,Er_field,nur_field]),
    v.Html(tag='div', class_='d-flex flex-row justify-space-around', children=[ec_field,Ec_field,nuc_field]),
    v.Html(tag='div', class_='d-flex flex-row justify-space-around', children=[f_field, k_field,kb_field]),
    v.Html(tag='div', class_='d-flex flex-row justify-space-around', children=[npts_field]),
])

# Buttons and field assembly
Assembly_button_fields_Layout = v.Html(tag='div', class_='d-flex flex-column', children=[
    v.Html(tag='div', class_='d-flex flex-row justify-space-around', children=[Buttons_Layout, Parameter_fields_Layout])
])

# Plot parameter pop-up display definition
plot_param_popup = v.Dialog(v_model=False, width='400',children=[v.Card(children=[
        v.Html(tag='div', children=[color_picker]),
        v.Html(tag='div', style_='width: 400px', class_='d-flex', children=[save_button,discard_button]),
    ])
])

# Display assembly definition
Content_layout = v.Html(tag='div', class_='d-flex flex-column', children=[Plot_layout,Assembly_button_fields_Layout])


# Footer
footer_layout = v.Footer(
    class_=" text-center d-flex flex-column",
    style_="background-color: white;",
    children=[
        v.Col(
            class_="text-center mt-4",
            children=[
                f"{datetime.now().year} — ",
                v.Html(tag="strong", children=['maxime.luyat@isae-supaero.fr']), 
                f" — ",
                v.Html(tag="strong", children=['GNU General Public License v3.0']),
            ],
        )
    ],
)

# Display the AppBar
app = v.App(children=[Menu_bar,Content_layout,v.Divider(vertical=False),footer_layout],style_='background-color: white; pa-0 ma-0;')

In [33]:
# Render the App
GR_active_button.fire_event('click', None)
display(app,plot_param_popup)

App(children=[AppBar(children=[ToolbarTitle(children=['SIMBBAAT'], layout=None, style_='font-size: 32px; font-…

Dialog(children=[Card(children=[Html(children=[ColorPicker(layout=None, width=400.0)], layout=None, tag='div')…